In [ ]:
from src.memory_game.game import Game
from src.strategy.strategies import BaseStrategy, RandomPerfectMemory, TileMemory
from src.models.cvmodel import CVModel
from src.models.memory import TileRNN

from fiftyone import ViewField as F

import math

import torch

import seaborn as sns

import pandas as pd

import numpy as np

import collections, gc, resource

from tqdm import tqdm

torch.no_grad()

In [2]:
print(f'Cuda available: {torch.cuda.is_available()}')
print(f'Cuda device count: {torch.cuda.device_count()}')

Cuda available: True
Cuda device count: 4


In [3]:
device = 3

device = torch.device(f"cuda:{device}")
print(torch.cuda.get_device_name(device))

GeForce RTX 2080 Ti


# Debug

In [4]:
def debug_memory():
    print('maxrss = {}'.format(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter(
        (str(o.device), str(o.dtype), tuple(o.shape))
        for o in gc.get_objects()
        if torch.is_tensor(o)
    )
    for line in sorted(tensors.items()):
        print('{}\t{}'.format(*line))


In [5]:
def print_mem():
    r = torch.cuda.memory_reserved(device)
    a = torch.cuda.memory_allocated(device)
    print(r-a, r, a)  # free inside reserved

# Global

In [6]:
n_trials = 20

size_range = [[3,3], [4,4], [5,5]]

# Functions

In [7]:
def play_strategy(strategy, game):
    win = False
    while not win:
        print('check1')
        strategy.pick()
        print('check2')
        win, tot = game.check_win()
        torch.cuda.empty_cache()
    return tot

In [8]:
def collect_data(n_trials, size_range, strategy, game):
    data = []
    for size in size_range:
        data_size = []
        for n in tqdm(range(n_trials), desc=f'Size: {size}'):
            game.set_size(size)
            strategy.reset()
            tot = play_strategy(strategy, game)
            data_size.append(math.log10(tot/np.prod(size)))
        data.append(data_size)
    return data

# Collect data

In [9]:
game = Game(
            dataset_name='coco-2017', 
            split='validation', 
            field="ground_truth", 
            ds_filter=F("supercategory").is_in(('person', 'animal', 'food', 'vehicle', 'indoor')),
            dataset_dir ="/home/ravi/datasets/coco-2017",
        )

Found annotations at '/home/ravi/datasets/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


/home/ravi/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [10]:
# bs = BaseStrategy(game)
# bs_data = collect_data(n_trials, size_range, bs, game)


In [11]:
# rpm = RandomPerfectMemory(game)
# rpm_data = collect_data(n_trials, size_range, rpm, game)


In [12]:
cvmodel = CVModel('pytorch/vision:v0.10.0', 'resnet18', model_dir='/home/ravi/models')
memory = TileRNN('lstm', 1000, 1000)
tm = TileMemory(game, cvmodel, memory, distance_metric='manhattan', device=device)
tm_data = collect_data(n_trials, size_range, tm, game)

Using cache found in /home/ravi/models/pytorch_vision_v0.10.0
Size: [3, 3]:   0%|          | 0/20 [00:00<?, ?it/s]

55324160 134217728 78893568
96756224 1107296256 1010540032
40438272 1973420032 1932981760
42181120 2898264064 2856082944
41873920 3821010944 3779137024
60205568 4762632192 4702426624
61948416 5687476224 5625527808
59544064 6608125952 6548581888
77875712 7549747200 7471871488
79618560 8474591232 8394972672
77214208 9395240960 9318026752


Size: [3, 3]:   0%|          | 0/20 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 3; 10.76 GiB total capacity; 9.53 GiB already allocated; 13.25 MiB free; 9.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
bs_df = pd.melt(pd.DataFrame(bs_data).T, var_name='size', value_name='nclicks')
bs_df['strategy'] = 'Random'

rpm_df = pd.melt(pd.DataFrame(rpm_data).T, var_name='size', value_name='nclicks')
rpm_df['strategy'] = 'Perfect memory'

tm_df = pd.melt(pd.DataFrame(tm_data).T, var_name='size', value_name='nclicks')
tm_df['strategy'] = 'LSTM'

In [ ]:
data = pd.concat([bs_df, rpm_df, tm_df])

# Plot

In [ ]:
sns.catplot(x="size", y='nclicks', hue="strategy", kind="point", 
            capsize=.15, data=data, aspect=1.5, errwidth=0.5)